In [41]:
import numpy as np
import pandas as pd
import torch
from allennlp.predictors.predictor import Predictor

In [42]:
predictor = Predictor.from_path("ser_dir_large_model/model.tar.gz")

Преобразуем данные из test.csv, парсим по колонкам

In [44]:
test_df =pd.read_csv('test.csv')[['Event','Xintent','Xemotion','Otheremotion']]
def str2list(x):
    return [y.strip() for y in x[2:-2].replace('"','').replace("'",'').split(',')]
for col in ['Xintent','Xemotion','Otheremotion']:
    test_df[col] = test_df[col].apply(str2list)

Группируем по фразам отдельно намерения, эмоции субъекта и эмоции окружающих

In [45]:
phrase2oemotion = {}
phrase2xemotion = {}
phrase2xintent = {}
res_xi = []
res_xe = []
res_oe = []

phrase_set = [x for x in test_df.Event]

for phrase in phrase_set:
    for x in test_df[test_df.Event == phrase].Xintent:
        res_xi += x
    phrase2xintent[phrase] = set(res_xi)
    res_xi.clear()
    for x in test_df[test_df.Event == phrase].Xemotion:
        res_xe += x
    phrase2xemotion[phrase] = set(res_xe)
    res_xe.clear()
    for x in test_df[test_df.Event == phrase].Otheremotion:
        res_oe += x
    phrase2oemotion[phrase] = set(res_oe)
    res_oe.clear()

Считаем recall следующим образом: из всех вариантов, выданных моделью для данной фразы, оставляем те, что есть в контрольном множестве фразы, сформированном ранее, и делим их количество на размер контрольного множества. Затем делим полученную сумму на количество фраз. Проделываем для каждого из выходов (intent, xreact и oreact)

In [46]:
res_oe = 0
res_xe = 0
res_xi = 0

for phrase in phrase_set:
    result = predictor.predict(source=phrase)

    oreact = set([" ".join(react) for react in result["oreact_top_k_predicted_tokens"]])    
    intents = set([" ".join(react) for react in result["xintent_top_k_predicted_tokens"]])
    xreact = set([" ".join(react) for react in result["xreact_top_k_predicted_tokens"]])

    res_oe += len(phrase2oemotion[phrase].intersection(oreact)) / len(phrase2oemotion[phrase])
    res_xe += len(phrase2oemotion[phrase].intersection(xreact)) / len(phrase2oemotion[phrase])
    res_xi += len(phrase2oemotion[phrase].intersection(intents)) / len(phrase2oemotion[phrase])

print("Recall for intents: ", res_xi / len(phrase_set))
print("Recall for x_react: ", res_xe / len(phrase_set))
print("Recall for o_react: ", res_oe / len(phrase_set))

Recall for intents:  0.4166370244707726
Recall for x_react:  0.5821430851597509
Recall for o_react:  0.6264394706359872
